In [1]:
# Instalação de bibliotecas necessárias

! pip3 install pandas
! pip3 install matplotlib
! pip3 install numpy
! pip3 install sklearn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no lon

In [2]:
# Importação de bibliotecas de uso geral

import pandas as pd
import sklearn
import statistics 
import matplotlib.pyplot as plt
from decimal import Decimal
import numpy as np

In [3]:
# Leitura do conjunto de dados

data = pd.read_csv('AVPdb.csv',sep=';')
display(data.head())

,Id,Sequence,Length,Virus,Abbreviation,Family,Nomenclature,Source,Uniprot,Cell_Line,Inhibition/IC50,Unit,Target,Assay,Accession,Title,Authors,Reference
0,AVP0001,PYVGSGLYRR,10,Avian myeloblastosis virus,AMV,Retroviridae,p2-p10,RSV gag polyprotein,P03354,-,50,μM,Release,Protease activity assay,1331099,Mechanism of inhibition of the retroviral prot...,"Cameron CE, Grinde B, Jentoft J, Leis J, Weber...",J Biol Chem. 1992
1,AVP0002,SMIENLEYM,9,Lymphocytic choriomeningitis virus,LCMV,Arenaviridae,-,Synthetic,NaN,MC57,50,μM,CTL lysis,In vitro cytotoxicity assay,1383569,Design of high-affinity major histocompatibili...,"Gairin JE, Oldstone MB.",J Virol. 1992
2,AVP0003,ECRSTSYAGAVVNDL,15,Herpes simplex virus 1,HSV 1,Herpesviridae,H2-(1-15),HSV ribonucleotide reductase subunit 2,P10224,BHK-21/C,42,μM,Replication,Ribonucleotide reductase assay,3040743,Structure-activity studies on synthetic peptid...,"Gaudreau P, Michaud J, Cohen EA, Langelier Y, ...",J Biol Chem. 1987
3,AVP0004,STSYAGAVVNDL,12,Herpes simplex virus 1,HSV 1,Herpesviridae,H2-(4-15),HSV ribonucleotide reductase subunit 2,P10224,BHK-21/C,29,μM,Replication,Ribonucleotide reductase assay,3040743,Structure-activity studies on synthetic peptid...,"Gaudreau P, Michaud J, Cohen EA, Langelier Y, ...",J Biol Chem. 1987
4,AVP0005,YAGAVVNDL,9,Herpes simplex virus 1,HSV 1,Herpesviridae,H2-(7-15),HSV ribonucleotide reductase subunit 2,P10224,BHK-21/C,60,μM,Replication,Ribonucleotide reductase assay,3040743,Structure-activity studies on synthetic peptid...,"Gaudreau P, Michaud J, Cohen EA, Langelier Y, ...",J Biol Chem. 1987


In [13]:
# Funções auxiliares para o processo

from sklearn.preprocessing import LabelEncoder

def get_subsequences(input_sequence, size):
    sequence = input_sequence.replace(' ', '')
    subsequences = []
    if len(sequence) < size:
        seq = sequence
        while(len(seq) < size):
            seq = seq + '0'
        subsequences.append(seq)
        return subsequences
    n_subsequences = len(sequence) - size + 1
    for i in range(n_subsequences):
        subsequences.append(sequence[i:i+size])
    return subsequences

def extract_aminoacid_symbols(data):
    symbols = ['0']
    for index, x in data.iterrows():
        x_sequence = x['Sequence'].replace(' ', '')
        for x_symbol in x_sequence:
            present = False
            for symbol in symbols:
                if x_symbol == symbol:
                    present = True
                    break
            if not present:
                symbols.append(x_symbol)
    encoder = LabelEncoder()
    encoder.fit(symbols)
    return symbols, encoder

def build_basic_dataframe(data, size):
    subsequences = []
    virus_ids = []
    virus_abbreviations = []
    sars_cov = []
    hcv = []
    for index, x in data.iterrows():
        abbreviation = x['Abbreviation']
        x_subsequences = get_subsequences(x['Sequence'], size)
        for x_subsequence in x_subsequences:
            subsequences.append(x_subsequence)
            virus_abbreviations.append(abbreviation)
            hcv.append(1 if abbreviation == 'HCV' else 0)
            sars_cov.append(1 if abbreviation == 'SARS-CoV' else 0)
    encoder = LabelEncoder()
    encoder.fit(virus_abbreviations)
    virus_ids = encoder.transform(virus_abbreviations)
    return pd.DataFrame(list(zip(subsequences, virus_ids, hcv, sars_cov)), columns =['subsequence', 'virus_id', 'hcv', 'sars_cov']), encoder

def build_encoder_translation_dataframe(data, size):
    df, virus_encoder = build_basic_dataframe(data, size)
    _, symbol_encoder = extract_aminoacid_symbols(data)
    translations = []
    for index, x in df.iterrows():
        subsequence = x['subsequence']
        subsequence_arr = []
        for s in subsequence:
           subsequence_arr.append(s) 
        encoded = symbol_encoder.transform(subsequence_arr)
        dictionary = {
            'virus_id': x['virus_id'],
            'sars_cov': x['sars_cov'],
            'hcv': x['hcv']
        }
        for i in range(size):
            dictionary['pos_'+str(i+1)] = encoded[i]
        translations.append(dictionary)
    return pd.DataFrame(translations), virus_encoder, symbol_encoder

def build_aminoacid_position_dataframe(data, size):
    df, virus_encoder = build_basic_dataframe(data, size)
    symbols, _ = extract_aminoacid_symbols(data)
    aminoacid_positions = []
    for index, x in df.iterrows():
        subsequence = x['subsequence']
        dictionary = {
            'virus_id': x['virus_id'],
            'sars_cov': x['sars_cov'],
            'hcv': x['hcv']
        }
        for i in range(len(subsequence)):
            if subsequence[i] != '0':
                dictionary[subsequence[i] + '_' + str(i+1)] = 1
            for s in symbols:
                if s != subsequence[i] and s != '0':
                    dictionary[s + '_' + str(i+1)] = 0
        aminoacid_positions.append(dictionary)
    return pd.DataFrame(aminoacid_positions), virus_encoder

,virus_id,sars_cov,hcv,P_1,Y_1,V_1,G_1,S_1,L_1,R_1,...,N_8,C_8,T_8,A_8,D_8,K_8,F_8,Q_8,H_8,W_8
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,41,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,41,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
df, virus_encoder, symbol_encoder = build_encoder_translation_dataframe(data, 8)
X = df.drop(columns=['virus_id', 'sars_cov', 'hcv'])

In [6]:
y = df['virus_id'].values.ravel()
clf = RandomForestClassifier(random_state=0)
statistics.fmean(cross_val_score(clf, X, y, cv=k_fold, n_jobs=1))

0.7178487168975394

In [7]:
y = df['sars_cov'].values.ravel()
clf = RandomForestClassifier(random_state=0)
statistics.fmean(cross_val_score(clf, X, y, cv=k_fold, n_jobs=1))

0.980060800301645

In [8]:
y = df['hcv'].values.ravel()
clf = RandomForestClassifier(random_state=0)
statistics.fmean(cross_val_score(clf, X, y, cv=k_fold, n_jobs=1))

0.9693983227523437